## Import modules

In [1]:
import numpy as np
from sklearn.linear_model import SGDRegressor, LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error
import os
import time

## Utils

In [2]:
def load_matrix(path: str, verbose: bool = True) -> np.ndarray:
    matrix = np.load(path)
    if verbose:
        print("Loaded matrix from", os.path.basename(path))
        print("Shape:", matrix.shape)
        print("Example:\n", matrix[:3], end="\n\n")
    return matrix

In [3]:
%cd ../../

/home/shkarupa/Documents/Semester6/Coursework/parallel-gradient-descent


## Generate data

In [4]:
!python demo/gen_data.py --help

Generate train and test data for linear regression model

Tool arguments:
  -h, --help          show this help message and exit
  -t , --n-train      number of train samples (default: 1000)
  -e , --n-eval       number of evaluation samples (default: 100)
  -f , --n-features   number of per sample features (default: 10)
  -s , --seed         seed that make data generation deterministic (default:
                      None)
  -o , --out-dir      path to directory where generated data will be stored
                      (default: ./data)


In [5]:
!python demo/gen_data.py --n-train 25000 --n-eval 2500 --n-features 25 --seed 42 --out-dir data

## Look at data

In [6]:
!ls data

x_eval.npy  x_train.npy  y_eval.npy  y_train.npy


In [7]:
x_train = load_matrix("data/x_train.npy")
y_train = load_matrix("data/y_train.npy")

x_eval = load_matrix("data/x_eval.npy")
y_eval = load_matrix("data/y_eval.npy")

Loaded matrix from x_train.npy
Shape: (25000, 25)
Example:
 [[-0.78876782 -0.93433388 -0.62381164  0.49449039  0.66985235 -2.34489026
   2.20601262  0.58554924 -0.99532385  1.06093379  0.06140756  0.10178809
  -0.7774324  -0.13746189  0.92637013 -1.08991456  0.90956384  0.378188
  -0.16182648 -0.51063043  0.08942337  1.72058511 -1.24840161 -0.55753184
   0.87106578]
 [ 0.15150384  1.86471513 -0.25839877  2.46021107  0.5243476  -0.99595698
  -2.4489341  -0.20082074 -0.10422274 -1.1890064   1.19507318 -0.19739341
   0.72453046 -0.83207458 -0.51136239  1.92516888  1.11091932 -0.29542615
   0.61680902  0.89166571 -1.03800368 -0.30955555 -1.26621354 -1.13170714
  -1.05167034]
 [ 1.3079373  -1.74548713  0.9587125   1.59040617 -0.26800735 -2.20166815
  -0.39878351  0.71667763  1.54870281 -0.71186963 -1.43268376 -0.90544979
  -0.82187994  0.73842441 -0.18159903  0.74039985  0.3462451  -0.26220498
   0.0891566  -0.00612928  0.05626937 -0.8788912   0.59366987  0.5744294
  -0.11859269]]

Loaded m

## Run Linear regression

In [8]:
!./build/bin/gradient_descent -h

Linear regression config:

CLI options:
  -h [ --help ]                        produce help message
  -c [ --config ] arg                  path to config file

Algorithm options:
  -i [ --input-path ] arg              path to input NPY file
  -t [ --target-path ] arg             path to target NPY file
  -e [ --eval-path ] arg               path to evaluation NPY file
  -o [ --out-path ] arg (=output.npy)  path to output NPY file
  -p [ --parallel ]                    wether to use parallel or serial SGD
  -n [ --num-epochs ] arg (=1000)      number of training epochs
  -l [ --lr ] arg (=0.001)             learning rate
  -w [ --weight-decay ] arg (=0.01)    L2 regularization lambda term
  --normalize                          wether to normalize input
  --num-threads arg (=11)              number of threads to use for parallel 
                                       SGD
  --num-step-epochs arg (=1)           number of epochs to compute in each 
                                       th

### SGD

In [9]:
!cat examples/serial_config.cfg

input-path = data/x_train.npy
target-path = data/y_train.npy
eval-path = data/x_eval.npy
out-path = output/serial_pred.npy
parallel = false
num-epochs = 10000
lr = 0.001
weight-decay = 0.01
normalize = true


In [10]:
!./build/bin/gradient_descent --config examples/serial_config.cfg

SGD: 3676 ms


In [11]:
y_pred_sgd = load_matrix("output/serial_pred.npy", verbose=False)

In [12]:
print("MSE:", mean_squared_error(y_pred_sgd.squeeze(), y_eval))

MSE: 16.18457694921623


### Parallel SGD

In [13]:
!cat examples/parallel_config.cfg

input-path = data/x_train.npy
target-path = data/y_train.npy
eval-path = data/x_eval.npy
out-path = output/parallel_pred.npy
parallel = true
num-epochs = 10000
lr = 0.001
weight-decay = 0.01
normalize = true
num-threads = 11
num-step-epochs = 100


In [14]:
!./build/bin/gradient_descent --config examples/parallel_config.cfg

SGD: 8245 ms


In [15]:
y_pred_parallel_sgd = load_matrix("output/parallel_pred.npy", verbose=False)

In [16]:
print("MSE:", mean_squared_error(y_pred_parallel_sgd, y_eval))

MSE: 16.18446840903993


### Sklearn

In [17]:
regressor = make_pipeline(StandardScaler(), LinearRegression())
# regressor = make_pipeline(
#     StandardScaler(), 
#     SGDRegressor(alpha=0.01, learning_rate="constant", eta0=0.001),
# )

In [18]:
# start = time.perf_counter_ns()
regressor.fit(x_train, y_train.squeeze())
# end = time.perf_counter_ns()
# print("SGD:", (end - start) / 1e6, "ms")

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('linearregression', LinearRegression())])

In [19]:
y_pred = regressor.predict(x_eval)

In [20]:
print("MSE:", mean_squared_error(y_pred, y_eval))

MSE: 16.184555636494746
